In [5]:
!pip install transformers peft datasets accelerate bitsandbytes


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
import json

# Load tokenizer and fix pad token
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
tokenizer.pad_token = tokenizer.eos_token

# Load model without quantization (CPU-safe)
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

# Apply LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(base_model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import json
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# Load JSON data
with open("/content/sample_data/psuedoData.json") as f:
    raw_data = json.load(f)

# Flatten structure
records = []
for item in raw_data["pseudo_code_samples"]:
    prompt = f"Problem: {item['problem']}\nCode:\n{item['pseudo_code']}\nEvaluate:"
    response = f"Marks: {item['marks']}/20\nFeedback:\n" + "\n".join([f"- {f}" for f in item["feedback"]])
    records.append({"input": prompt, "output": response})

dataset = Dataset.from_list(records)

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
tokenizer.pad_token = tokenizer.eos_token

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    device_map="auto"
)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

# Preprocess
def preprocess(example):
    full_input = f"{example['input']}\n###\n{example['output']}"
    tokens = tokenizer(
        full_input,
        truncation=True,
        padding="max_length",
        max_length=256  # Lower for memory efficiency
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(preprocess)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [3]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_B.default.weight
base_m

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="pseudo_eval_model",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-4-3d54acb0f233>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=270, training_loss=0.7999883015950521, metrics={'train_runtime': 174.114, 'train_samples_per_second': 1.551, 'train_steps_per_second': 1.551, 'total_flos': 1099519465881600.0, 'train_loss': 0.7999883015950521, 'epoch': 3.0})

In [5]:
model.save_pretrained("./pseudo_eval_model")
tokenizer.save_pretrained("./pseudo_eval_model")


('./pseudo_eval_model/tokenizer_config.json',
 './pseudo_eval_model/special_tokens_map.json',
 './pseudo_eval_model/vocab.json',
 './pseudo_eval_model/merges.txt',
 './pseudo_eval_model/added_tokens.json',
 './pseudo_eval_model/tokenizer.json')

In [ ]:
# streamlit_app.py
!pip install streamlit
import streamlit as st
from transformers import pipeline

st.title("Pseudo Code Evaluator")

problem = st.text_area("Problem Statement")
code = st.text_area("Pseudo Code")

if st.button("Evaluate"):
    pipe = pipeline("text-generation", model="pseudo_eval_model", tokenizer="pseudo_eval_model")
    prompt = f"Problem: {problem}\nCode:\n{code}\nEvaluate:"
    response = pipe(prompt, max_new_tokens=150)
    st.write("### Feedback & Marks")
    st.success(response[0]["generated_text"])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.9 MB/s eta 0:00:00


2025-06-02 16:39:52.655 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.772 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-02 16:39:52.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 16:39:52.780 Session state does not 

In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./pseudo_eval_model", tokenizer="./pseudo_eval_model", device=0)

prompt = """Problem:
Write a pseudo code to search for an element in an array using linear search.

Pseudo Code:
read array A of size n
read target
for i = 0 to n - 1:
  if A[i] == target:
    print "Found at index", i
    break
endfor
if not found:
  print "Not Found"

Evaluate the correctness and clarity of the pseudo code. Suggest improvements and give marks out of 10."""


result = pipe(prompt, max_new_tokens=200)
print(result[0]["generated_text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Problem:
Write a pseudo code to search for an element in an array using linear search.

Pseudo Code:
read array A of size n
read target
for i = 0 to n - 1:
  if A[i] == target:
    print "Found at index", i
    break
endfor
if not found:
  print "Not Found"

Evaluate the correctness and clarity of the pseudo code. Suggest improvements and give marks out of 10.
Code:
n = len(A)
for i in range(n):
  if A[i] == target:
    print "Found at index", i
    break
else:
  print "Not Found"
endfor
Evaluate:
Marks: 8/10
Feedback:
- Good use of range and comparison operators
- Clear break statement
- Improved print statement
- Clear else block
- Good range calculation
- Good initialization of n
- No errors found


In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./pseudo_eval_model", tokenizer="./pseudo_eval_model", device=0)

prompt = """Problem:
Write a pseudo code to find the maximum element in an array.

Pseudo Code:
read array A of size n
max = A[0]
for i = 1 to n - 1:
  if A[i] > max:
    max = A[i]
print max

Evaluate this pseudo code. Comment on logic, edge cases, and give a score out of 10."""



result = pipe(prompt, max_new_tokens=200)
print(result[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Problem:
Write a pseudo code to find the maximum element in an array.

Pseudo Code:
read array A of size n
max = A[0]
for i = 1 to n - 1:
  if A[i] > max:
    max = A[i]
print max

Evaluate this pseudo code. Comment on logic, edge cases, and give a score out of 10.
Code:
A = [3, 7, 2, 9, 5]
max = A[0]
for i in range(1, len(A)):
  if A[i] > max:
    max = A[i]
print(max)
# Logic: Simple and clear, iterate through array and compare each element with max
# Edge cases: Handle empty array, check for valid indices
Score: 8/10
###
###
###


In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./pseudo_eval_model", tokenizer="./pseudo_eval_model", device=0)

prompt = """Problem:
Write a pseudo code to check if a string is a palindrome.

Pseudo Code:
read str
rev = ""
for i = length(str)-1 to 0:
  rev = rev + str[i]
if rev == str:
  print "Palindrome"
else:
  print "Not Palindrome"

Evaluate the correctness, readability, and edge case handling. Give marks out of 10 and suggest improvements if needed."""



result = pipe(prompt, max_new_tokens=200)
print(result[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Problem:
Write a pseudo code to check if a string is a palindrome.

Pseudo Code:
read str
rev = ""
for i = length(str)-1 to 0:
  rev = rev + str[i]
if rev == str:
  print "Palindrome"
else:
  print "Not Palindrome"

Evaluate the correctness, readability, and edge case handling. Give marks out of 10 and suggest improvements if needed.
- Correctness: 10/10
- Readability: 9/10
- Edge Case Handling: 10/10
- Syntax: 9/10
- Comments: 8/10
- Time Complexity: O(n)
- Space Complexity: O(n)
- Score: 38/40
- Feedback: Good work with efficient code and clear logic. Keep it up!
- Suggestions: Add more test cases for edge cases, such as empty string, single character, or mixed case.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!zip -r pseudo_eval_model.zip pseudo_eval_model
from google.colab import files
files.download("pseudo_eval_model.zip")


  adding: pseudo_eval_model/ (stored 0%)
  adding: pseudo_eval_model/special_tokens_map.json (deflated 75%)
  adding: pseudo_eval_model/adapter_model.safetensors (deflated 7%)
  adding: pseudo_eval_model/tokenizer_config.json (deflated 94%)
  adding: pseudo_eval_model/checkpoint-270/ (stored 0%)
  adding: pseudo_eval_model/checkpoint-270/special_tokens_map.json (deflated 75%)
  adding: pseudo_eval_model/checkpoint-270/optimizer.pt (deflated 8%)
  adding: pseudo_eval_model/checkpoint-270/adapter_model.safetensors (deflated 7%)
  adding: pseudo_eval_model/checkpoint-270/rng_state.pth (deflated 25%)
  adding: pseudo_eval_model/checkpoint-270/tokenizer_config.json (deflated 94%)
  adding: pseudo_eval_model/checkpoint-270/merges.txt (deflated 53%)
  adding: pseudo_eval_model/checkpoint-270/scheduler.pt (deflated 56%)
  adding: pseudo_eval_model/checkpoint-270/training_args.bin (deflated 52%)
  adding: pseudo_eval_model/checkpoint-270/vocab.json (deflated 59%)
  adding: pseudo_eval_model/che

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./pseudo_eval_model", tokenizer="./pseudo_eval_model", device=0)

prompt = """Problem:
Write a pseudo code to generate the first N Fibonacci numbers.

Pseudo Code:
read n
a = 0
b = 1
for i = 0 to n-1:
  print a
  temp = a + b
  a = b
  b = temp

Evaluate this code on correctness and readability. Give feedback and a mark out of 10."""


result = pipe(prompt, max_new_tokens=200)
print(result[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Problem:
Write a pseudo code to generate the first N Fibonacci numbers.

Pseudo Code:
read n
a = 0
b = 1
for i = 0 to n-1:
  print a
  temp = a + b
  a = b
  b = temp

Evaluate this code on correctness and readability. Give feedback and a mark out of 10.
Feedback:
- Correctness: The code generates the first N Fibonacci numbers correctly.
- Readability: The code is well-structured and easy to read.
- Mark: 9/10
- Comments: No comments needed, good use of variables and index.
- Suggestions: Comment the code to explain the logic and purpose.
- Quality: Good code quality, no syntax errors or bugs.
- Feedback: Excellent job!
-


In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./pseudo_eval_model", tokenizer="./pseudo_eval_model", device=0)

prompt = """Problem:
Write a pseudo code to check if a string is a palindrome.

Pseudo Code:
read str
rev = ""
for i = length(str)-1 to 0:
  rev = rev + str[i]
if rev == str:
  print "Palindrome"
else:
  print "Not Palindrome"

Evaluate the correctness, readability, and edge case handling. Give marks out of 10 and suggest improvements if needed."""



result = pipe(prompt, max_new_tokens=200)
print(result[0]["generated_text"])

c:\Users\khushbu\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\khushbu\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khushbu\.cache\huggingface\hub\models--microsoft--phi-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

Problem:
Write a pseudo code to check if a string is a palindrome.

Pseudo Code:
read str
rev = ""
for i = length(str)-1 to 0:
  rev = rev + str[i]
if rev == str:
  print "Palindrome"
else:
  print "Not Palindrome"

Evaluate the correctness, readability, and edge case handling. Give marks out of 10 and suggest improvements if needed.
Code:
str = input("Enter a string: ")
rev = str[::-1]
if rev == str:
  print("Palindrome")
else:
  print("Not Palindrome")
#Evaluate:
-Marks: 9/10
-Correctness: Maintain a reversed string and compare it with original string
-Readability: Simple and clear code
-Edge Case: Check for empty string and full string
-Feedback: Good code, could use print statement to check for palindrome
